 mistral & qlora r=64
 alpha=16
 learning_rate=1e-4
threshold = 0.75  
 question: title +selftext
 answer:falcon_summary

In [ ]:
import torch

# Clear GPU cache
torch.cuda.empty_cache()


In [ ]:
import gc

# Trigger garbage collection
gc.collect()


0

# Install packages

In [ ]:
!pip install git+https://github.com/huggingface/accelerate


  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-3mo5hr1k
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-3mo5hr1k
  Resolved https://github.com/huggingface/accelerate to commit cd7df4117d92f660965d5f737364395b5693a535
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q peft bitsandbytes wandb

In [ ]:
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c


  Cloning https://github.com/huggingface/trl.git (to revision 7630f877f91c556d9e5a3baa4b6e2894d90ff84c) to /tmp/pip-req-build-029zdbq4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-029zdbq4
  Running command git rev-parse -q --verify 'sha^7630f877f91c556d9e5a3baa4b6e2894d90ff84c'
  Running command git fetch -q https://github.com/huggingface/trl.git 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Running command git checkout -q 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Resolved https://github.com/huggingface/trl.git to commit 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
 #!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 wandb

# Authenticate with wandb, gdrive

In [ ]:
import wandb

# Import packages

In [ ]:
# Import necessary packages for the fine-tuning process
import os                          # Operating system functionalities
import torch                       # PyTorch library for deep learning
from datasets import load_dataset,DatasetDict,Dataset  # Loading datasets for training
from transformers import (
    AutoModelForCausalLM,          # AutoModel for language modeling tasks
    AutoTokenizer,                # AutoTokenizer for tokenization
    BitsAndBytesConfig,           # Configuration for BitsAndBytes
    HfArgumentParser,             # Argument parser for Hugging Face models
    TrainingArguments,            # Training arguments for model training
    pipeline,                     # Creating pipelines for model inference
    logging,                      # Logging information during training
)
from peft import LoraConfig, PeftModel  # Packages for parameter-efficient fine-tuning (PEFT)
from trl import SFTTrainer         # SFTTrainer for supervised fine-tuning
from datetime import datetime
import pandas as pd

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!huggingface-cli whoami

sherrys


Input huggingface token, no need to Add token as git credential? n

# Initialize model and dataset by name

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

#model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# The instruction dataset to use
#dataset_name = "chriztopherton/Reddit_RAFT"

# Fine-tuned model name
new_model = "mistral-2-7b_qlora_falcon_426"

#new_model = "mistral-2-7b-reddit_qlora_falcon"

# Define LLM model parameters

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64 # used to be 64


# Alpha parameter for LoRA scaling
lora_alpha = 16        # used to be 16 #let's try r256,alpha512 next

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

In [ ]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
#original code: output_dir = "./chris_ft/ft_v2"
output_dir = "./426_mistral_qlora"

In [ ]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Load dataset and train/val/test split

In [ ]:
pwd

'/'

/


In [ ]:
import pickle

with open("417_datasplit_0.75train_test_dataset.pkl","rb") as f:
    train_test_dataset = pickle.load(f)

with open("417_datasplit_0.75test_valid_dataset.pkl","rb") as f:
    test_valid_dataset = pickle.load(f)

with open("417_datasplit_0.75train_test_valid_dataset.pkl","rb") as f:
    train_test_valid_dataset = pickle.load(f)

## Continue from here

In [ ]:
train_test_valid_dataset['train'][0]

{'title': 'Whats your favorite and least favorite InterstateUS Route?',
 'selftext': 'My favorite would be tied between I in ColoradoUtah, US in Yellowstone, M in the UP of Michigan, I in west Montana My least favorite is definitely US between Toledo and Flint. I in Michigan, I between Memphis and Little Rock, II in Ohio and I in Virginia are honorable mentions',
 'comments': 'As a former trucker having been in every nook and cranny. my favorite section of interstate is the Columbia River Gorge along I Oregon Washington line. Absolutely gorgeous. Such a lovely drive. edit to add least favorite II across Texas. Barren it seems like it will never end., from LA to Jacksonville favorite. through Nebraska and Iowa, makes me wish I flew., Fav UT Hwy through Grand Staircase, US Hwy through AZUT, Any of the highways in the black hills Least Fav US in OK interstate level truck traffic passing through towns, Least fave I from outside Topeka to outskirts of Denver. That western half of damn near 

def transform_data(example):
    ques = example['selftext']
    ans = example['comments']
    return {
        "text": f"<s>[INST] {ques} [/INST] {ans} </s>"
    }

transformed_dataset = train_test_valid_dataset.map(transform_data)
transformed_dataset

In [ ]:
def transform_data(example):
    ques = example['title']+ " " + example['selftext']
    ans = example['falcon_summary']
    return {
        "text": f"<s>[INST] {ques} [/INST] {ans} </s>"
    }

transformed_dataset = train_test_valid_dataset.map(transform_data)
transformed_dataset

Map: 100%|██████████| 14/14 [00:00<00:00, 3459.22 examples/s]


DatasetDict({
    train: Dataset({
        features: ['title', 'selftext', 'comments', 'falcon_summary', 'title_comments_dot', 'questions_comments_dot', 'title_query_comments_dot', 'title_query_falcon_dot', 'falcon_title_dot', 'falcon_questions_dot', 'falcon_comments_dot', '__index_level_0__', 'text'],
        num_rows: 257
    })
    test: Dataset({
        features: ['title', 'selftext', 'comments', 'falcon_summary', 'title_comments_dot', 'questions_comments_dot', 'title_query_comments_dot', 'title_query_falcon_dot', 'falcon_title_dot', 'falcon_questions_dot', 'falcon_comments_dot', '__index_level_0__', 'text'],
        num_rows: 15
    })
    valid: Dataset({
        features: ['title', 'selftext', 'comments', 'falcon_summary', 'title_comments_dot', 'questions_comments_dot', 'title_query_comments_dot', 'title_query_falcon_dot', 'falcon_title_dot', 'falcon_questions_dot', 'falcon_comments_dot', '__index_level_0__', 'text'],
        num_rows: 14
    })
})

In [ ]:
transformed_dataset['train'][0]

{'title': 'Whats your favorite and least favorite InterstateUS Route?',
 'selftext': 'My favorite would be tied between I in ColoradoUtah, US in Yellowstone, M in the UP of Michigan, I in west Montana My least favorite is definitely US between Toledo and Flint. I in Michigan, I between Memphis and Little Rock, II in Ohio and I in Virginia are honorable mentions',
 'comments': 'As a former trucker having been in every nook and cranny. my favorite section of interstate is the Columbia River Gorge along I Oregon Washington line. Absolutely gorgeous. Such a lovely drive. edit to add least favorite II across Texas. Barren it seems like it will never end., from LA to Jacksonville favorite. through Nebraska and Iowa, makes me wish I flew., Fav UT Hwy through Grand Staircase, US Hwy through AZUT, Any of the highways in the black hills Least Fav US in OK interstate level truck traffic passing through towns, Least fave I from outside Topeka to outskirts of Denver. That western half of damn near 

# Define training procedures

In [ ]:
# Step 2 :Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
# Step 3 :Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [ ]:
# Step 4 :Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]


In [ ]:
# Step 5 :Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Step 6 :Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
project = "mistral qlora_4-26_0.75"
base_model_name = "mistral"
run_name = base_model_name + "-" + project

# Number of training epochs
num_train_epochs = 30

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
#learning_rate = 1e-4 #for llama qlora

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

evaluation_strategy = "steps"

eval_steps = 10 #earlier 25

# Save checkpoint every X updates steps
save_steps = 10 #eraler 50

# Log every X updates steps
logging_steps = 10

## eval steps, save steps and logging steps
## original 25,50,25
## try 50,100,50

In [ ]:
# Step 7 :Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    evaluation_strategy = evaluation_strategy,
    eval_steps=eval_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    report_to="wandb",
    #report_to="tensorboard",
    #run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
)

In [ ]:
#%load_ext tensorboard

# Step 8 :Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset['train'],
    eval_dataset=transformed_dataset['valid'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
from transformers import EarlyStoppingCallback
# Step 8 :Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset['train'],
    eval_dataset=transformed_dataset['valid'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # can try 5
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:228: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 14/14 [00:00<00:00, 3221.96 examples/s]


# Train

In [ ]:
# Step 9 :Train model
trainer.train()

# Step 10 :Save trained model
trainer.model.save_pretrained(new_model)

wandb: Currently logged in as: bertha-tam (298bwanderchat). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
10,3.724000,3.475307
20,3.522400,2.766788
30,2.828900,2.618817
40,2.734100,2.540861
50,2.694400,2.477295
60,2.556800,2.422192
70,2.489200,2.414394
80,2.134800,2.491447
90,2.168000,2.486903
100,2.131600,2.490754


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

In [ ]:
torch.cuda.empty_cache()
import gc
#del variables
gc.collect()

8972

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   5098 MiB |  14702 MiB | 186934 GiB | 186929 GiB |\n|       from large pool |   4393 MiB |  13996 MiB | 186772 GiB | 186768 GiB |\n|       from small pool |    705 MiB |    706 MiB |    161 GiB |    160 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   5098 MiB |  14702 MiB | 186934 GiB | 186929 GiB |\n|       from large pool |   4393 MiB |  13996 MiB |

In [ ]:
!nvidia-smi



Wed Apr  3 05:04:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |  15005MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.71s/it]


In [ ]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]
model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]


model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 1.35M/4.54G [00:00<05:36, 13.5MB/s]
model-00001-of-00003.safetensors:   0%|          | 1.40M/4.94G [00:00<05:53, 14.0MB/s]


model-00002-of-00003.safetensors:   0%|          | 1.47M/5.00G [00:00<05:39, 14.7MB/s]

model-00003-of-00003.safetensors:   0%|          | 4.98M/4.54G [00:00<02:48, 26.9MB/s]
model-00001-of-00003.safetensors:   0%|          | 4.85M/4.94G [00:00<03:09, 26.0MB/s]


model-00002-of-00003.safetensors:   0%|          | 5.05M/5.00G [00:00<03:04, 27.1MB/s]

model-00003-of-00003.safetensors:   0%|          | 8.81M/4.54G [00:00<02:21, 32.0MB/s]
model-00001-of-00003.safetensors:   0%|          | 7.45M/4.94G [00:00<03:16, 25.2MB/s]




CommitInfo(commit_url='https://huggingface.co/sherrys/mistral-2-7b_qlora_falcon_426/commit/ed284d7706e6d7395820ef4f5abe16a1310a2d8b', commit_message='Upload tokenizer', commit_description='', oid='ed284d7706e6d7395820ef4f5abe16a1310a2d8b', pr_url=None, pr_revision=None, pr_num=None)